In [1]:
%%time
%matplotlib inline
import importlib
import new_import  
importlib.reload(new_import)
from new_import import *

CPU times: user 8.24 s, sys: 6.08 s, total: 14.3 s
Wall time: 8.33 s


In [ ]:
cluster, client = notebook_utils.initialize_dask(use_gateway=True, workers=(1,10))
# Khai báo 1 Datacube là dc
dc = datacube.Datacube()
client

In [ ]:
date_range = ('2022-09-01', '2023-10-01')
longtitude_range = (105.5, 106.4)
latitude_range = (9.2, 10.0) 

coordinates = (longtitude_range, latitude_range)

In [ ]:
dsvh, dsvv = load_data_sen1(dc, date_range, coordinates)
data_sen2 = load_data_sen2(dc, date_range, coordinates)
notebook_utils.heading(notebook_utils.xarray_object_size(data_sen2))
display(data_sen2)

In [ ]:
average_vv = calculate_average(dsvv, time_pattern='1M')
progress(average_vv)

In [ ]:
average_vh = calculate_average(dsvh, time_pattern='1M')
progress(average_vh)

In [ ]:
# Tiến hành loại bỏ các vị trí bị mây ảnh hưởng
result = mask_cloud(data_sen2)
progress(result)

In [ ]:
# calculate NDVI
dsNDVI = calculate_indices(result, index='NDVI', satellite_mission='s2')
ndvi = dsNDVI["NDVI"]

In [ ]:
# calculate average NDVI
average_ndvi = calculate_average(ndvi,time_pattern="1M")
progress(average_ndvi)

In [ ]:
%%time
average_vh.compute()
average_vv.compute()
average_ndvi.compute()

In [ ]:
plt.imshow(average_ndvi.isel(time=6), vmin=None, vmax=None)
average_ndvi.shape

SAVE THE DATA FOR TRAINING SERVERAL TIME

In [ ]:
np.savez('input/saved_datasets/full_dataset_cloud.npz', ndvi=average_ndvi, vh=average_vh, vv=average_vv)

LOAD SAVED DATASET TO USE

In [2]:
full_dataset = np.load('input/saved_datasets/full_dataset_cloud.npz')

In [3]:
ndvi = full_dataset['nvdi']
vh = full_dataset['vh']
vv = full_dataset['vv']

In [4]:
mask = ~np.isnan(ndvi)
X_train = np.stack([vh[mask], vv[mask]], axis=1)
y_train = ndvi[mask]

In [5]:
# np.savez('datasets/train_dataset/full_data_regressor_train.npz', data=X_train, label=y_train)

In [6]:
num_samples = 2000000

# Generate random indices
random_indices = np.random.choice(X_train.shape[0], size=num_samples, replace=False)

# Subset the data using the random indices
X_train_subset = X_train[random_indices]
y_train_subset = y_train[random_indices]

# Check the new shape
print(X_train_subset.shape)
print(y_train_subset.shape)


(2000000, 2)
(2000000,)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_train_subset, y_train_subset, test_size=0.2, random_state=42)

TRAIN MODEL

In [8]:
%%time
best_model, best_params, best_score, cv_results = find_best_regression(X_train, 
                                                                       y_train, regressors['random_forest'], 
                                                                       param_grids_regressor['random_forest'])

# Print best parameters, best result, and MSE over splits
print(f"Best parameters: {best_params}")
print(f"Best result (negative MSE): {-best_score}")

Best parameters: {'max_depth': 10, 'n_estimators': 1000}
Best result (negative MSE): 0.04785161659200724
CPU times: user 49min 18s, sys: 8.22 s, total: 49min 27s
Wall time: 2h 25min 54s


In [9]:
folder_path = 'output/regressors/'
file_path = os.path.join(folder_path, 'rf_cloud.pkl')
joblib.dump(best_model, file_path)

['output/regressors/rf_cloud.pkl']

In [10]:
y_pred = best_model.predict(X_test)
# Calculate R² score
r2 = r2_score(y_test, y_pred)
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"R² Score: {r2}")
print(f"Mean Squared Error: {mse}")

R² Score: 0.7433191566743431
Mean Squared Error: 0.047650972371530896


FILL CLOUD 

In [ ]:
best_model = joblib.load("output/regressors/rf_cloud.pkl")

In [ ]:
filled_ndvi = np.copy(ndvi)

In [ ]:
X_pred = np.stack([vh[~mask], vv[~mask]], axis=1)
filled_ndvi[~mask] = best_model.predict(X_pred)

In [ ]:
filled_ndvi[~mask]

In [ ]:
np.savez('input/saved_datasets/rf_full_dataset_filled.npz', ndvi=filled_ndvi, vv=vv, vh=vh)

In [ ]:
client.close()
cluster.close()